# 1. Setting

In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
# 필요한 라이브러리 세팅
from selenium import webdriver
import time
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# 2. 데이터 크롤링

In [ ]:
# 카테고리별 url
category_list = ['https://www.musinsa.com/category/001001?d_cat_cd=001001&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={}&display_cnt=90&sale_goods=&group_sale=&ex_soldout=&color={}&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure=',
      'https://www.musinsa.com/category/001010?d_cat_cd=001010&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={}&display_cnt=90&sale_goods=&group_sale=&ex_soldout=&color={}&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure=',
      'https://www.musinsa.com/category/001011?d_cat_cd=001011&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={}&display_cnt=90&sale_goods=&group_sale=&ex_soldout=&color={}&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure=',
      'https://www.musinsa.com/category/001002?d_cat_cd=001002&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={}&display_cnt=90&sale_goods=&group_sale=&ex_soldout=&color={}&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure=',
      'https://www.musinsa.com/category/001003?d_cat_cd=001003&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={}&display_cnt=90&sale_goods=&group_sale=&ex_soldout=&color={}&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure=',
      'https://www.musinsa.com/category/001005?d_cat_cd=001005&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={}&display_cnt=90&sale_goods=&group_sale=&ex_soldout=&color={}&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure=',
      'https://www.musinsa.com/category/001004?d_cat_cd=001004&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={}&display_cnt=90&sale_goods=&group_sale=&ex_soldout=&color={}&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure=',
      'https://www.musinsa.com/category/001006?d_cat_cd=001006&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={}&display_cnt=90&sale_goods=&group_sale=&ex_soldout=&color={}&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure='
      ]

df_name_list = ['shortsleeve', 'longsleeve', 'sleeveless', 'shirt',
                'collar', 'sweat', 'hoodie', 'sweater']

In [ ]:
def get_musinsa_data(li):
  # 사전 세팅
  global musinsa_df
  musinsa_url = []
  result = {}
  color = {
    3 : '회색',
    11 : '빨간색',
    10 : '분홍색',
    12 : '주황색',
    9 : '노란색',
    6 : '녹색',
    7 : '파란색',
    8 : '보라색',
    4 : '갈색',
    29 : '샌드'
  }

  for key in color: # 1 페이지만
    musinsa_url.append(li.format(1, key))

  for i in range(len(musinsa_url)):
    # url 연결 및 태그 가져오기
    driver.get(musinsa_url[i])
    time.sleep(3) # 3초 대기 - HTTPConnectionPool 오류를 해결하기 위함

    pageString = driver.page_source
    soup = BeautifulSoup(pageString, features="html.parser")
    if soup.find(name='div', attrs = {'class': 'error-network-text'}):
      continue
    
    # 필요한 데이터만 가져오기 위한 준비
    list_box = soup.find(name='div', attrs = {'class': 'list-box box'}).find_all('li',{'class':'li_box'})
    if len(list_box) == 0:
      continue
    else:
      for j in range(0, len(list_box)):
        try:
          ## 브랜드명
          brand_name = list_box[j].find('p', attrs = {'class' : 'item_title'}).text
          ## 판매가격 (list - 정가, 할인가)
          price = list_box[j].find('p', attrs = {'class' : 'price'}).text.replace(' ', '').replace('\n', '').split('원')[:2]
          ## url, 의류명, 썸네일
          clothes_info3 = str(list_box[j].find('a', attrs = {'class' : 'img-block'})) # 세 정보가 들어있는 태그 -> string화
          ### 1) url
          url = clothes_info3.split('href=')[1].split('"')[1]
          ### 2) 의류명
          clothes_name = clothes_info3.split('title=')[1].split(">")[0].replace('"','')
          ### 3) 썸네일
          thumbnail = clothes_info3.split('data-original=')[1].split(' ')[0].replace('"', '').replace('_125', '_500')
          ## 색상명
          color = musinsa_url[i].split('color=')[1].split('&')[0]

          result[url] = [brand_name, clothes_name, price[0], price[1], thumbnail, color]
      
        except(ValueError, AttributeError, IndexError, TypeError):
          pass

  # 데이터 프레임화
  musinsa_df = pd.DataFrame(result)
  musinsa_df = musinsa_df.transpose()

  return musinsa_df

In [ ]:
for i in tqdm(range(len(category_list))):
  get_musinsa_data(category_list[i])
  musinsa_df = musinsa_df.reset_index()
  musinsa_df.rename(columns={'index' : 'url'}, inplace=True)
  musinsa_df['category'] = df_name_list[i]
  musinsa_df.columns = ['url', 'brand', 'name', 'price', 'discount_price', 'thumbnail', 'color', 'category']
  musinsa_df.to_csv('/content/drive/MyDrive/MUSINSA/data/'+ df_name_list[i] + '.csv', encoding='utf-8-sig')

musinsa_df

100%|██████████| 8/8 [10:26<00:00, 78.35s/it]


,url,brand,name,price,discount_price,thumbnail,color,category
0,https://store.musinsa.com/app/goods/1599217,테이크이지,램스울 크루넥 오버니트(연차콜),"54,000","37,800",https://image.msscdn.net/images/goods_img/2020...,3,sweater
1,https://store.musinsa.com/app/goods/1723790,라퍼지스토어,울 오버 헨리넥 니트_Gray,"68,000","45,900",https://image.msscdn.net/images/goods_img/2020...,3,sweater
2,https://store.musinsa.com/app/goods/1736057,테이크이지,램스울 크루넥 오버니트(헤더그레이),"54,000","37,800",https://image.msscdn.net/images/goods_img/2021...,3,sweater
3,https://store.musinsa.com/app/goods/2226767,드로우핏,오버사이즈 집업 카라 터틀넥 니트 [M.GREY],"76,000","68,400",https://image.msscdn.net/images/goods_img/2021...,3,sweater
4,https://store.musinsa.com/app/goods/1191494,테이크이지,램스울 크루넥 오버니트(그레이),"54,000","37,800",https://image.msscdn.net/images/goods_img/2019...,3,sweater
...,...,...,...,...,...,...,...,...
895,https://store.musinsa.com/app/goods/1609018,컨셉원,050907 캐시미어 블렌드 스웨터 (10050907),"89,800","38,000",https://image.msscdn.net/images/goods_img/2020...,29,sweater
896,https://store.musinsa.com/app/goods/2258364,암피스트,BEIGE BIG POCKET SWEATER,"139,000","97,300",https://image.msscdn.net/images/goods_img/2021...,29,sweater
897,https://store.musinsa.com/app/goods/1727926,가인드,BASIC LOGO KNIT,"89,000","40,000",https://image.msscdn.net/images/goods_img/2020...,29,sweater
898,https://store.musinsa.com/app/goods/2218879,베르노,21FW_16gauge Mock-neck_Knit_Light-Khaki,"128,000",,https://image.msscdn.net/images/goods_img/2021...,29,sweater


In [ ]:
df_name_list = ['shortsleeve', 'longsleeve', 'sleeveless', 'shirt',
                'collar', 'sweat', 'hoodie', 'sweater']

# 개별 데이터 저장
shortsleeve_df = pd.read_csv('/content/drive/MyDrive/MUSINSA/data/' + df_name_list[0] + '.csv')
longsleeve_df = pd.read_csv('/content/drive/MyDrive/MUSINSA/data/' + df_name_list[1] + '.csv')
sleeveless_df = pd.read_csv('/content/drive/MyDrive/MUSINSA/data/' + df_name_list[2] + '.csv') #533
shirt_df = pd.read_csv('/content/drive/MyDrive/MUSINSA/data/' + df_name_list[3] + '.csv')
collar_df = pd.read_csv('/content/drive/MyDrive/MUSINSA/data/' + df_name_list[4] + '.csv') #695
sweat_df = pd.read_csv('/content/drive/MyDrive/MUSINSA/data/' + df_name_list[5] + '.csv')
hoodie_df = pd.read_csv('/content/drive/MyDrive/MUSINSA/data/' + df_name_list[6] + '.csv')
sweater_df = pd.read_csv('/content/drive/MyDrive/MUSINSA/data/' + df_name_list[7] + '.csv')

In [ ]:
# 전체 의류 데이터 저장
final_musinsa = pd.concat([shortsleeve_df, longsleeve_df, sleeveless_df, shirt_df, collar_df, sweat_df, hoodie_df, sweater_df])
final_musinsa = final_musinsa[['url', 'brand', 'name', 'price', 'discount_price', 'thumbnail', 'color', 'category']]
final_musinsa.to_csv('/content/drive/MyDrive/MUSINSA/data/clothes.csv', encoding='utf-8-sig')

# 3. Thumbnail 이미지 저장하기

In [ ]:
import urllib.request
import time
from PIL import Image
import pandas as pd

musinsa_url = '/content/drive/MyDrive/MUSINSA/data/clothes.csv'
musinsa_df = pd.read_csv(musinsa_url)
musinsa_df = musinsa_df.drop([musinsa_df.columns[0]], axis='columns')

for j in tqdm(range(len(musinsa_df))):
  # 다운받을 썸네일(이미지) url
  thumbnail_url = musinsa_df['thumbnail'][j] 

  # 이미지 요청 및 다운로드
  thumbnail_name = musinsa_df['url'][j].split('/')[-1] + ".jpg"
  urllib.request.urlretrieve(thumbnail_url, '/content/drive/MyDrive/MUSINSA/thumbnail/' + thumbnail_name)

100%|██████████| 6628/6628 [1:22:54<00:00,  1.33it/s]


In [ ]:
import os

image_list = os.listdir('/content/drive/MyDrive/MUSINSA/thumbnail')
len(image_list)

6628